In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import subprocess
import numpy as np
import random
from copy import deepcopy as cc
from transformers import BertTokenizer, BertModel
import gc
from torch import nn
from torch.nn import GELU as GELU
import argparse
from dataset import *
gc.disable()
gc.collect()
import pdb

- $ a = \begin{matrix}  0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 0 & 0 \\ \end{matrix} $ &#8594
$ b = \begin{matrix}  1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \\ \end{matrix} $ &#8594 
$ c = \begin{matrix}  0 & 0 & 0 & 0 \\ 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ \end{matrix} $ &#8594
$ tri\_matrix = \begin{matrix}  1 & 1 & 1 & 1 \\ 0 & 1 & 1 & 1 \\ 0 & 0 & 1 & 1 \\ 0 & 0 & 0 & 1 \\ \end{matrix} $
$ (a + c) = \begin{matrix} 0 & 1 & 0 & 0 \\ 1 & 0 & 1 & 0 \\ 0 & 1 & 0 & 1 \\ 0 & 0 & 1 & 0 \\ \end{matrix} $

- $ SCORE =  \begin{matrix}  
\langle Q_0, K_0 \rangle & \dots & \langle Q_0, K_3 \rangle\\ \vdots & \ddots & \\ \langle Q_3, K_0 \rangle &  & \langle Q_3, K_3 \rangle \\ 
\end{matrix}
= \begin{matrix} S_{00} & \dots & S_{03} \\ \vdots & \ddots & \\S_{30} & & S_{33} \\ \end{matrix} $

- $ SCORE  \xrightarrow[- \infty]{MASK(a+c)} 
\begin{matrix} 
-\infty & S_{01} & -\infty & -\infty \\ S_{10} & -\infty & S_{12} & -\infty \\ -\infty & S_{21} & -\infty & S_{23} \\ -\infty & -\infty & S_{32} & -\infty \\ \end{matrix} 
\xrightarrow[]{SOFTMAAX(dim = 0)} 
\begin{matrix} 
0 & P_{01} & 0 & 0 \\ P_{10} & 0 & P_{12} & 0 \\ 0 & P_{21} & 0 & P_{23} \\ 0 & 0 & P_{32} & 0 \\ \end{matrix}  
\xrightarrow[]{Score^T\dot (Score+\epsilon)} 
\begin{matrix} 
0 & \sqrt{\langle P_{01}, P_{01} +\epsilon \rangle} & 0 & 0 \\ 
\sqrt{\langle P_{10},P_{01} +\epsilon \rangle} & 0 & \sqrt{\langle P_{12},P_{21} +\epsilon \rangle} & 0 \\ 
0 & \sqrt{\langle P_{21},P_{12} +\epsilon \rangle }& 0 & \sqrt{\langle P_{23},P_{32} +\epsilon \rangle} \\ 
0 & 0 & \sqrt{\langle P_{32},P_{23} +\epsilon \rangle} & 0 \\ \end{matrix}$

- $\hat{A^l} \sim
\begin{matrix} 
0 & \sqrt{\langle P_{01}, P_{01}\rangle} & 0 & 0 \\ 
\sqrt{\langle P_{10}P_{01} \rangle} & 0 & \sqrt{\langle P_{12}P_{21} \rangle} & 0 \\ 
0 & \sqrt{\langle P_{21}P_{12} \rangle }& 0 & \sqrt{\langle P_{23}P_{32} \rangle} \\ 
0 & 0 & \sqrt{\langle P_{32}P_{23} \rangle}& 0 \\ \end{matrix}
\equiv
\begin{matrix} 
0 & \hat{a_0} & 0 & 0  \\ 
\hat{a_0} & 0 & \hat{a_1} & 0 \\ 
0 & \hat{a_1} & 0 & \hat{a_2} \\ 
0 & 0 & \hat{a_2} & 0 \\ \end{matrix}$

- $A^l = A^{l-1} + (1 - A^{l-1})\hat{A^l} = 
\begin{matrix} 0 & a_0 & 0 & 0  \\ a_0 & 0 & a_1 & 0 \\ 0 & a_1 & 0 & a_2 \\ 0 & 0 & a_2 & 0 \\ \end{matrix}
\xrightarrow[0]{MASK(a)} 
\begin{matrix} 0 & log(a_0 + \epsilon) & 0 & 0  \\ 0 & 0 & log(a_1 + \epsilon) & 0 \\ 0 & 0 & 0 & log(a_2 + \epsilon) \\ 0 & 0 & 0 & 0 \\ \end{matrix}
\xrightarrow[]{C.tri\_matrix}
\begin{matrix} 
0 & log(a_0 + \epsilon) & log(a_0+ \epsilon) & log(a_0+ \epsilon)  \\ 
0 & 0 & log(a_1+ \epsilon) & log(a_1+ \epsilon) \\ 
0 & 0 & 0 & log(a_2 + \epsilon) \\ 
0 & 0 & 0 & 0 \\ \end{matrix}
\xrightarrow[tri\_matrix^T.C]{log()} \sim
\begin{matrix} 
0 &  log(a_0 + \epsilon) & log(a_0)+log(a_1) & log(a_0)+log(a_1)+log(a_2)  \\ 
0 & 0 & log(a_1) & log(a_1)+log(a_2) \\ 
0 & 0 & 0 & log(a_2) \\ 
0 & 0 & 0 & 0 \\ 
\end{matrix}$

- $ \equiv 
\begin{matrix} 0 & log(a_0) & log(a_0a_1) & log(a_0a_1a_2)  \\ 
0 & 0 & log(a_1) & log(a_1a_2) \\ 
0 & 0 & 0 & log(a_2) \\ 
0 & 0 & 0 & 0 \\ 
\end{matrix}
\xrightarrow[]{exp()}
\begin{matrix} 0 & C_{01} & C_{02} & C_{02}  \\ 0 & 0 & C_{12} & C_{13} \\ 0 & 0 & 0 & C_{23} \\ 0 & 0 & 0 & 0 \\ \end{matrix}
\xrightarrow[]{ + C^T}
\begin{matrix} 0 & C_{01} & C_{02} & C_{02}  \\ 
C_{01} & 0 & C_{12} & C_{13} \\ 
C_{02} & C_{12} & 0 & C_{23} \\ 
C_{03} & C_{13} & C_{23} & 0 \\ 
\end{matrix}$

In [155]:
class Config():
    "Configuration for BERT model"
    vocab_size: int = 30522             # Size of Vocabulary
    dim: int = 768                      # Dimension of Hidden Layer in Transformer Encoder
    n_layers: int = 12                  # Numher of Hidden Layers
    n_heads: int = 12                   # Numher of Heads in Multi-Headed Attention Layers
    dim_ff: int = 768*4                 # Dimension of Intermediate Layers in Positionwise Feedforward Net
    activ_fn: str = "gelu"              # Non-linear Activation Function Type in Hidden Layers
    p_drop_hidden: float = 0.1          # Probability of Dropout of various Hidden Layers
    p_drop_attn: float = 0.1            # Probability of Dropout of Attention Layers
    max_len: int = 512                  # Maximum Length for Positional Embeddings
    n_segments: int = 2                 # Number of Sentence Segments
    layer_norm_eps: int = 1e-12         # eps value for the LayerNorms
    output_attentions : bool = False    # Weather to output the attention scores

config = Config()
INFINITY = 1e15
EPSILON = 1e-15


+ #  <u>Embedding Layer</u>

In [156]:
class BertEmbeddings(nn.Module):
    "The embedding module from word, position and token_type embeddings."
    def __init__(self, cfg):
        super().__init__()
        self.word_embeddings = nn.Embedding(cfg.vocab_size, cfg.dim, padding_idx=0) # token embedding
        self.position_embeddings = nn.Embedding(cfg.max_len, cfg.dim) # position embedding
        self.token_type_embeddings = nn.Embedding(cfg.n_segments, cfg.dim) # segment(token type) embedding

        self.LayerNorm = nn.LayerNorm(cfg.dim, eps=cfg.layer_norm_eps)
        self.dropout = nn.Dropout(cfg.p_drop_hidden)

    def forward(self, x, seg):
        # X -> B x S
        # seg -> B x S

        S = x.size(1)
        pos = torch.arange(S, dtype=torch.long, device=x.device)
        pos = pos[None, :].expand_as(x) # (S,) -> (B, S)

        e = self.word_embeddings(x) + self.position_embeddings(pos) + self.token_type_embeddings(seg)
        e = self.LayerNorm(e)
        e = self.dropout(e)
        return e

* # <u>Self Attention Layer</u>

In [181]:
class GroupAttention2(nn.Module):
    def __init__(self, cfg):
        super(GroupAttention2, self).__init__()
        self.d_model = cfg.dim
        self.linear_key = nn.Linear(cfg.dim, cfg.dim)
        self.linear_query = nn.Linear(cfg.dim, cfg.dim)
        self.LayerNorm = nn.LayerNorm(cfg.dim, eps= cfg.layer_norm_eps)
        self.dropout = nn.Dropout(cfg.p_drop_attn)

        torch.nn.init.xavier_normal_(self.linear_key.weight)
        torch.nn.init.zeros_(self.linear_key.bias)

        torch.nn.init.xavier_normal_(self.linear_query.weight)
        torch.nn.init.zeros_(self.linear_query.bias)

        torch.nn.init.uniform_(self.LayerNorm.weight)
        torch.nn.init.zeros_(self.LayerNorm.bias)

    def forward(self, hidden_states, attention_mask, ip_event_loc, ip_event_mask, prior_A):
        ''' 
        hidden_states = B x S x D
        attention_mask = B x S(V/K)
        A_prior = B x S(Q) x S(V/K)
        '''

        B, S_full = hidden_states.size()[:2]
        hidden_event_subset = hidden_states[np.arange(B)[:, None], ip_event_loc]
        context = self.LayerNorm(hidden_event_subset)
        B, S = hidden_event_subset.size()[:2]

        a = torch.from_numpy(np.diag(np.ones(S - 1, dtype=np.int32),1)).to(hidden_states.device)
        b = torch.from_numpy(np.diag(np.ones(S, dtype=np.int32),0)).to(hidden_states.device)
        c = torch.from_numpy(np.diag(np.ones(S - 1, dtype=np.int32),-1)).to(hidden_states.device)
        tri_matrix = torch.from_numpy(np.triu(np.ones([S, S], dtype=np.float32),0)).to(hidden_states.device)

        a_full = torch.from_numpy(np.diag(np.ones(S_full - 1, dtype=np.int32),1)).to(hidden_states.device)
        b_full = torch.from_numpy(np.diag(np.ones(S_full, dtype=np.int32),0)).to(hidden_states.device)
        c_full = torch.from_numpy(np.diag(np.ones(S_full - 1, dtype=np.int32),-1)).to(hidden_states.device)
        tri_matrix_full = torch.from_numpy(np.triu(np.ones([S_full, S_full], dtype=np.float32),0)).to(hidden_states.device)

        # mask -> BxSxS
        mask = ip_event_mask[:, None, :] & (a+c)[None, :, :]
        
        key = self.linear_key(context)
        query = self.linear_query(context)
        scores = torch.matmul(query, key.transpose(-2, -1)) / (self.d_model / 2)
        scores = scores.masked_fill(mask == 0, (-1 * INFINITY))

        # Compute the A for the [et_sep] locations 
        # A -> B x S x S
        A = F.softmax(scores, dim=-1)
        A = torch.sqrt(A * A.transpose(-2,-1) + EPSILON)

        # A -> B x (S-1) ; Extract the diagonal(offset 1) elements from the A
        A = torch.cat([torch.diagonal(A[batch], 1)[None, :] for batch in range(B)])

        # scatter the above A matrix in the A_full (B x S_full) matrix
        A_new = cc(attention_mask).float()
        A_new = A_new.scatter(1, ip_event_loc[:, :-1], A.float())

        A_new = A_new[:, :-1]
        A_new= [torch.diag(A_new[batch, :], 1)[None, :, :] for batch in range(B)]
        A_new = torch.cat(A_new)
        A_new += A_new.transpose(-1, -2).contiguous()
        A_new = prior_A + (1. - prior_A)*A_new

        t = torch.log(A_new + EPSILON).masked_fill(a_full==0, 0).matmul(tri_matrix_full)
        C_prior = tri_matrix_full.matmul(t).exp().masked_fill((tri_matrix_full.int()-b_full)==0, 0)
        C_prior = C_prior + C_prior.transpose(-2, -1).contiguous() + torch.from_numpy(np.diag(np.ones(S_full)))
        
        return C_prior.float(), A_new.float()

In [182]:
class GroupAttention(nn.Module):
    def __init__(self, cfg):
        super(GroupAttention, self).__init__()
        self.d_model = cfg.dim
        self.linear_key = nn.Linear(cfg.dim, cfg.dim)
        self.linear_query = nn.Linear(cfg.dim, cfg.dim)
        self.LayerNorm = nn.LayerNorm(cfg.dim, eps= cfg.layer_norm_eps)
        self.dropout = nn.Dropout(cfg.p_drop_attn)

    def forward(self, hidden_states, attention_mask, prior_A):
        ''' 
        dim( hidden_states ) = B x S x D
        dim( attention_mask ) = B x S(V/K)
        dim( prior ) = S(Q) x S(V/K)
        '''
        B, S = hidden_states.size()[:2]

        context = self.LayerNorm(hidden_states)
        a = torch.from_numpy(np.diag(np.ones(S - 1, dtype=np.int32),1))
        b = torch.from_numpy(np.diag(np.ones(S, dtype=np.int32),0))
        c = torch.from_numpy(np.diag(np.ones(S - 1, dtype=np.int32),-1))
        tri_matrix = torch.from_numpy(np.triu(np.ones([S, S], dtype=np.float32),0))

        #mask = eos_mask & (a+c) | b
        mask = attention_mask[:, None, :] & (a+c)[None, :, :]
        
        key = self.linear_key(context)
        query = self.linear_query(context)
        
        scores = torch.matmul(query, key.transpose(-2, -1)) / (self.d_model / 2)
        
        scores = scores.masked_fill(mask == 0, -1e9)
        A = F.softmax(scores, dim=-1)
        A = torch.sqrt(A * A.transpose(-2,-1) + 1e-9)
        A = prior_A + (1. - prior_A)*A

        t = torch.log(A + 1e-9).masked_fill(a==0, 0).matmul(tri_matrix)
        C_prior = tri_matrix.matmul(t).exp().masked_fill((tri_matrix.int()-b)==0, 0)    
        C_prior = C_prior + C_prior.transpose(-2, -1) + torch.from_numpy(np.diag(np.zeros(S)))
        
        return C_prior.float(), A.float()

In [183]:

# X -> self_attn -> X [calculated by self attention]
class BertSelfAttention(nn.Module):
    """ Multi-Headed Dot Product Attention """
    def __init__(self, cfg):
        super().__init__()
        self.query = nn.Linear(cfg.dim, cfg.dim)
        self.key = nn.Linear(cfg.dim, cfg.dim)
        self.value = nn.Linear(cfg.dim, cfg.dim)
        self.dropout = nn.Dropout(cfg.p_drop_attn)
        self.n_heads = cfg.n_heads

    def forward(self, x, attention_mask = None, output_attentions = False, C_prior = None):
        """
        x, q(query), k(key), v(value) : (B(batch_size), S(seq_len), D(dim))
        mask : (B(batch_size) x S(seq_len))
        * split D(dim) into (H(n_heads), W(width of head)) ; D = H * W
        attention_mask_dim = B x S (dim S is for every value in the "x"), so need to repeat it for every query
        """
        B, S, D = x.shape
        H = self.n_heads
        W = int( D/H )
        assert W * H == D

        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q, k, v = self.query(x), self.key(x), self.value(x)
        q, k, v = q.reshape((B, S, H, W)), k.reshape((B, S, H, W)), v.reshape((B, S, H, W))
        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

        # (B, H, S(Q), W) @ (B, H, W, S(K/V)) -> (B, H, S(Q), S(K/V)) -Masking -> softmax-> (B, H, S(Q), S(K/V))
        attn_scores = q @ k.transpose(-2, -1) / np.sqrt(k.size(-1))
        
        # Set the attention score at places where MASK = 0 to very low value (-1e9)
        if attention_mask is not None:
            attn_scores = attn_scores.masked_fill(attention_mask[:, None, None, :] == 0, (-1 * INFINITY))
        attn_scores = self.dropout(F.softmax(attn_scores, dim=-1))
    
        # C_prior -> B x S x S
        if C_prior != None:
            attn_scores = attn_scores * C_prior[:, None, :, :]

        # (B, H, S, S) @ (B, H, S, W) -> (B, H, S, W) -trans-> (B, S, H, W) -merge-> (B, S, D)
        hidden_states = (attn_scores @ v).transpose(1, 2).contiguous()
        hidden_states = hidden_states.reshape(B, S, D)

        result = {}
        result['hidden_states'] = hidden_states 
        result['attn_scores'] =  None
        if output_attentions :
            result['attn_scores'] =  attn_scores

        return result


In [184]:
#  f(X) -> Linear (D => D) -> Dropout -> X1
#  X, f(X) -> LayerNorm( X + X1 )
class BertSelfOutput(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.dense = nn.Linear(cfg.dim, cfg.dim)
        self.LayerNorm = nn.LayerNorm(cfg.dim, eps= cfg.layer_norm_eps)
        self.dropout = nn.Dropout(cfg.p_drop_hidden)

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states

In [185]:
# X -> self_attn -> f(X) -> Linear (D => D) -> Dropout -> X1
# Y = LayerNorm( X + X1 )
class BertAttention(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.self = BertSelfAttention(cfg)
        self.output = BertSelfOutput(cfg)

    def forward(self, hidden_states, attention_mask=None, output_attentions = False, C_prior = None):
        self_outputs = self.self(hidden_states,  attention_mask, output_attentions, C_prior)
        attention_output = self.output(self_outputs['hidden_states'], hidden_states)

        outputs = {'hidden_states': attention_output, 'attn_scores': self_outputs['attn_scores']}
        return outputs        

* # <u>Forward Expansion layer</u>

In [186]:
# f(X) -> Linear(D => 4xD) -> GELU() -> X2
class BertIntermediate(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.dense = nn.Linear(cfg.dim, cfg.dim_ff)
        self.GELU = nn.GELU()

    def forward(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.GELU(hidden_states)
        return hidden_states

# f(X) -> Linear(4xD => D) -> Drouput -> X1
#  X, f(X) -> LayerNorm( X + X1 )
class BertOutput(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.dense = nn.Linear(cfg.dim_ff, cfg.dim)
        self.LayerNorm = nn.LayerNorm(cfg.dim, eps=cfg.layer_norm_eps)
        self.dropout = nn.Dropout(cfg.p_drop_hidden)

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states

* # <u>Defining single BERT-encoder layer</u>

In [187]:
class BertLayer(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.group_attention = GroupAttention2(cfg)
        self.attention = BertAttention(cfg)
        self.intermediate = BertIntermediate(cfg)
        self.output = BertOutput(cfg)

    def forward(self, hidden_states, ip_event_loc, ip_event_mask, attention_mask=None, output_attentions = False, A_prior = None):
        # pdb.set_trace()
        C_prior = None

        if A_prior != None:
            # C_prior, A_prior = self.group_attention(hidden_states, attention_mask, A_prior)
            C_prior, A_prior = self.group_attention(hidden_states = hidden_states, ip_event_loc = ip_event_loc, 
                                                    attention_mask = attention_mask, ip_event_mask = ip_event_mask, prior_A = A_prior)

        self_attention_outputs = self.attention(hidden_states, attention_mask, output_attentions, C_prior)
        intermediate_output = self.intermediate(self_attention_outputs['hidden_states'])
        layer_output = self.output(intermediate_output, self_attention_outputs['hidden_states'])

        outputs = {'hidden_states':layer_output, 'attn_scores': self_attention_outputs['attn_scores'], 
                  'A_prior': A_prior, 'C_prior': C_prior}
        return outputs

* # <u>Defining the BERT model</u>

In [188]:
class BertEncoder(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layer = nn.ModuleList([BertLayer(cfg) for _ in range(cfg.n_layers)])

    def forward(self, hidden_states, ip_event_loc, ip_event_mask, attention_mask=None, output_attentions = False, 
                output_hidden_states = False, A_prior = None, output_A_prior = False, output_C_prior = False):   

        all_hidden_states = [] if output_hidden_states else None
        all_self_attentions = [] if output_attentions else None
        all_A_priors = [] if output_A_prior else None
        all_C_priors = [] if output_C_prior else None

        for i, layer_module in enumerate(self.layer):
            if output_hidden_states:
                all_hidden_states.append(hidden_states)

            layer_outputs = layer_module(hidden_states, ip_event_loc, ip_event_mask, attention_mask, output_attentions, A_prior)
            hidden_states, attn_scores = layer_outputs['hidden_states'], layer_outputs['attn_scores'], 
            C_prior, A_prior = layer_outputs['C_prior'], layer_outputs['A_prior']

            if output_attentions:
                all_self_attentions.append(attn_scores)
            if output_A_prior:
                all_A_priors.append(A_prior)
            if output_C_prior:
                all_C_priors.append(C_prior)

        outputs = {'hidden_states': hidden_states, 'attn_scores': all_self_attentions, 
                  'all_hidden_states': all_hidden_states, 'all_A_priors': all_A_priors, 'all_C_priors': all_C_priors}
        return outputs 


class my_BERT(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.embeddings = BertEmbeddings(cfg)
        self.encoder = BertEncoder(cfg)
    
    def forward(self, x, seg, ip_event_loc, ip_event_mask, attention_mask=None, output_attentions = False, 
                output_hidden_states = False, A_prior = None, output_A_prior = False, output_C_prior = False):

        embedding_output = self.embeddings(x, seg)
        encoded_output = self.encoder(hidden_states = embedding_output, 
                                      attention_mask = attention_mask, 
                                      ip_event_loc = ip_event_loc, 
                                      ip_event_mask = ip_event_mask,
                                      output_attentions = output_attentions, 
                                      output_hidden_states = output_hidden_states,
                                      A_prior = A_prior,
                                      output_A_prior = output_A_prior,
                                      output_C_prior = output_C_prior)

        return encoded_output

In [189]:
config = Config()
MY_BERT = my_BERT(config)
BERT = BertModel.from_pretrained('bert-base-uncased')
MY_BERT.load_state_dict(BERT.state_dict(), strict = False);

In [190]:
MY_BERT_params = set([n for n, _ in MY_BERT.named_parameters()])
BERT_params = set([n for n, _ in BERT.named_parameters()])
print(f'Parametes in the model that are not in BERT:: {BERT_params.difference(MY_BERT_params)}')

Parametes in the model that are not in BERT:: {'pooler.dense.bias', 'pooler.dense.weight'}


In [167]:
dataset = tree2bert_dataset(data_path = './../Data/raw/seq2seq/train_short_prefix.txt.val',
                            max_seq_len = 256,
                            max_ev_len = 20)

fast_tokenizer = BertWordPieceTokenizer(vocab='./vocab/vocab.txt')
fast_tokenizer.add_special_tokens(['[et_sep]', '[ea_sep]', '[ds_sep]'])
fast_tokenizer.enable_truncation(max_length = 256)
fast_tokenizer.enable_padding(length=256)

In [191]:
# emb_layer = BertEmbeddings(config)
# att_layer = BertSelfAttention(config)
# grp_att_layer = GroupAttention2(config)

ip_text_tok = dataset[:2]['text_tok_src'].long()
ip_text_mask = dataset[:2]['text_mask_src'].long()
ip_event_loc = dataset[:2]['event_loc_src'].long()
ip_event_mask = dataset[:2]['event_mask_src'].long()

In [192]:
def get_initial_A(ip_text_mask, ip_event_loc):

    A_initial = cc(ip_text_mask)*1.0

    # Set all the locations with [et_sep] tokens as 0.0
    A_initial = A_initial.scatter(1, ip_event_loc.long(), torch.zeros_like(ip_event_loc).float())

    # exclude the last token as this vector will be a diagonal with offset 1
    A_initial = A_initial[:, :-1]

    # Create a diagonal matrix with this A as the diagonal at offset 1
    A_initial= [torch.diag(A_initial[i, :].float(), 1)[None, :, :] for i in range(A_initial.shape[0])]
    A_initial = torch.cat(A_initial)

    # Add it's transpose, as the A matrix is supposed to be symmetric
    A_initial += A_initial.transpose(-1, -2).contiguous()
    return A_initial

In [194]:
A_initial = get_initial_A(ip_text_mask, ip_event_loc)
result = MY_BERT(x = ip_text_tok, 
                seg = torch.ones_like(ip_text_tok), 
                ip_event_loc = ip_event_loc, 
                ip_event_mask = ip_event_mask, 
                attention_mask = ip_text_mask, 
                output_attentions = False, 
                output_hidden_states = False, 
                A_prior = A_initial, 
                output_A_prior = False,
                output_C_prior = True
                )

In [205]:
class TreeBERT2BERT(nn.Module):

    def __init__(self, config):
        super(TreeBERT2BERT, self).__init__()
        self.config = config
        self.encoder = Tree_BERT(self.config)

        BERT = BertModel.from_pretrained('bert-base-uncased')
        self.encoder.load_state_dict(BERT.state_dict(), strict = False);

        del BERT
        self.decoder = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased', 'bert-base-uncased').decoder
    
    def forward(self, src_text_tok, src_text_mask, src_event_loc, src_event_pad_mask, tar_text_tok, tar_pad_mask):

        # indices with label -100 will be ignored in the loss computation
        decoder_label = cc(tar_text_tok)
        decoder_label[tar_text_tok == 0] = -100

        # Get the encoded input
        A_initial = self.get_initial_A(self, ip_text_mask = src_text_mask, ip_event_loc = src_event_loc)
        encoded_src = self.encoder(x = src_text_tok, 
                                   seg = torch.ones_like(src_text_tok), 
                                   ip_event_loc = src_event_loc, 
                                   ip_event_mask = src_event_pad_mask, 
                                   attention_mask = src_text_mask, 
                                   output_attentions = False, 
                                   output_hidden_states = False, 
                                   A_prior = A_initial, 
                                   output_A_prior = False,
                                   output_C_prior = True)
        
        # Pass the encoded input to the decoder
        decoder_out = self.decoder(input_ids = tar_text_tok,
                            attention_mask = tar_pad_mask,
                            encoder_hidden_states = encoded_src['hidden_states'],
                            encoder_attention_mask = src_text_mask,
                            labels = decoder_label)
        return  decoder_out

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 0.7064, 0.7064, 0.7064, 0.7064, 0.7064, 0.7064, 0.7064,
        0.7064, 0.7064, 0.7064, 0.7064, 0.7064, 0.7064, 0.7064, 0.7064, 0.7064,
        0.7064, 0.7064, 0.7064, 0.7064, 0.3498, 0.3498, 0.3498, 0.3498, 0.3498,
        0.3498, 0.3498, 0.3498, 0.3498, 0.1767, 0.1767, 0.1767, 0.1767, 0.1767,
        0.1767, 0.1767, 0.1767, 0.1767, 0.1767, 0.1767, 0.1767, 0.1767, 0.1767,
        0.1767, 0.1767, 0.1767, 0.1767, 0.1767, 0.1767],
       grad_fn=<SliceBackward>)

In [204]:
ip_event_loc[0]

tensor([ 10,  30,  39,  59,  68,  94, 104, 106, 107, 108, 109, 110, 111, 112,
        113, 114, 115, 116, 117, 118])